In [ ]:
%config InteractiveShell.ast_node_interactivity='last_expr_or_assign'  # always print last expr.
%config InlineBackend.figure_format = 'svg'
%load_ext autoreload
%autoreload 2
%matplotlib inline

import logging

logging.basicConfig(level=logging.DEBUG)

In [ ]:
from tsdm.utils import LazyDict

In [ ]:
d = LazyDict({1: 1})
d.get(1)

In [ ]:
d

In [ ]:
import os
import zipfile
import requests
from pathlib import Path

In [ ]:
import tsdm

In [ ]:
from tsdm.models.pretrained import LinODEnet

In [ ]:
d = {}
?d.get

In [ ]:
LinODEnet.available_checkpoints()

In [ ]:
LinODEnet.get_checkpoints()

In [ ]:
tsdm.utils._utils.repackage_zip("20.zip")

In [ ]:
Path("30.zip").rename("20.zip")

In [ ]:
zf = zipfile.ZipFile("20.zip")
zf

In [ ]:
p = zf.infolist()[1]

In [ ]:
Path("asfagfs/2022-12-01-20.zip").stem

In [ ]:
url = "https://tubcloud.tu-berlin.de/s/P7SAkkaeGtAWJ2L/download?path=/LinODEnet/2022-12-01/20"
local_filename = "2022-12-01-20.zip"

r = requests.get(url, stream=True)

with open(local_filename, "wb") as f:
    for chunk in r.iter_content(chunk_size=1024):
        if chunk:  # filter out keep-alive new chunks
            f.write(chunk)

with zipfile.ZipFile(local_filename, "r") as zip_ref:
    zip_ref.extractall(".")

In [ ]:
import tsdm

In [ ]:
cls = tsdm.models.pretrained.LinODEnet

In [ ]:
pretrained = cls.from_remote_checkpoint("2022-12-01/20")

In [ ]:
pretrained.rawdata_path

In [ ]:
# cls.available_checkpoints()
model = cls.from_path("2022-12-01-linodenet-50-f79c5e885f2182835e7b88ba3205fb33.zip")
model.components["model"]

In [ ]:
pretrained = cls.from_remote_checkpoint("2022-12-01/30")

In [ ]:
model.components["model"]

In [ ]:
import os

In [ ]:
os.path.isdir("2022-12-01-linodenet-50-f79c5e885f2182835e7b88ba3205fb33.zip")

In [ ]:
str(model.rawdata_path)

In [ ]:
model.components["model"]